In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pydrake.all import (
    AddContactMaterial,
    AddMultibodyPlant,
    Box,
    Sphere,
    ConstantVectorSource,
    Cylinder,
    Capsule,
    CoulombFriction,
    ConstantValueSource,
    DeformableBodyConfig,
    DiagramBuilder,
    DiscreteDerivative,
    DifferentialInverseKinematicsIntegrator,
    DifferentialInverseKinematicsParameters,
    Filament,
    GeometryInstance,
    IllustrationProperties,
    Meshcat,
    MeshcatVisualizer,
    LeafSystem,
    MeshcatVisualizerParams,
    ProximityProperties,
    MultibodyPlantConfig,
    Parser,
    PdControllerGains,
    Rgba,
    PidController,
    PiecewisePose,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    SceneGraphConfig,
    RigidTransform,
    RotationMatrix,
    Multiplexer,
    MultibodyPlant,
    Simulator,
    VectorLogSink,
    Value,
    SpatialInertia,
)

In [ ]:
meshcat = Meshcat()
meshcat.SetProperty("/Axes", "visible", False)

In [ ]:
class PoseTrajectorySource(LeafSystem):
    def __init__(self, traj: PiecewisePose):
        super().__init__()
        self.traj_ = traj
        self.DeclareAbstractOutputPort(
            name="y0",
            alloc=lambda: Value(RigidTransform()),
            calc=self.CalcOutput
        )

    def CalcOutput(self, context, output):
        output.set_value(self.traj_.GetPose(context.get_time()))

In [ ]:
def PoseController():
    time_step = 2e-3  # 500 Hz
    plant = MultibodyPlant(0.0)
    Parser(plant).AddModelsFromUrl(
        "package://drake_models/franka_description/urdf/panda_arm.urdf")
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetRigidBodyByName("panda_link0").body_frame(),
        RigidTransform()
    )
    plant.Finalize()

    builder = DiagramBuilder()
    ik = builder.AddNamedSystem("ik", DifferentialInverseKinematicsIntegrator(
        robot=plant,
        frame_E=plant.GetRigidBodyByName("panda_link7").body_frame(),
        time_step=time_step,
        parameters=DifferentialInverseKinematicsParameters(7, 7)
    ))
    deriv = builder.AddSystem(DiscreteDerivative(num_inputs=7, time_step=time_step))
    mux = builder.AddSystem(Multiplexer([7, 7]))
    constant_true = builder.AddSystem(ConstantValueSource(Value(True)))
    builder.Connect(ik.get_output_port(), deriv.get_input_port())
    builder.Connect(ik.get_output_port(), mux.get_input_port(0))
    builder.Connect(deriv.get_output_port(), mux.get_input_port(1))
    builder.Connect(constant_true.get_output_port(), ik.GetInputPort("use_robot_state"))
    builder.ExportInput(ik.GetInputPort("X_AE_desired"), "X_AE")
    builder.ExportInput(ik.GetInputPort("robot_state"), "state")
    builder.ExportOutput(mux.get_output_port(), "desired_state")
    return builder.Build()

In [ ]:
def AddWorkpiece(plant):
    model_instance = plant.AddModelInstance("workpiece")
    density = 500
    color = [0.1, 0.1, 0.1, 1]
    friction = CoulombFriction(0.2, 0.2)
    kZ = [0, 0, 1]

    size = [0.05, 0.05, 0.006]
    base = plant.AddRigidBody("base", model_instance, SpatialInertia.SolidBoxWithDensity(density, *size))
    plant.SetDefaultFreeBodyPose(base, RigidTransform([0.5, 0.0, 0.003]))
    plant.RegisterVisualGeometry(base, RigidTransform(), Box(*size), "base", color)
    plant.RegisterCollisionGeometry(base, RigidTransform(), Box(*size), "base", friction)

    size = [0.005, 0.12]
    trunk = plant.AddRigidBody("trunk", model_instance, SpatialInertia.SolidCapsuleWithDensity(density, *size, kZ))
    plant.WeldFrames(base.body_frame(), trunk.body_frame(), RigidTransform([0.0, 0.0, 0.063]))
    plant.RegisterVisualGeometry(trunk, RigidTransform(), Capsule(*size), "trunk", color)
    plant.RegisterCollisionGeometry(trunk, RigidTransform(), Capsule(*size), "trunk", friction)

    sizes = [0.0035, 0.1]
    tf = [
        RigidTransform(RotationMatrix.MakeZRotation(np.pi/4) @ RotationMatrix.MakeYRotation(np.pi/2), [0.0, 0.0, 0.12]),
        RigidTransform(RotationMatrix.MakeZRotation(-np.pi/4) @ RotationMatrix.MakeYRotation(np.pi/2), [0.0, 0.0, 0.12]),
        RigidTransform(RotationMatrix.MakeYRotation(np.pi/2), [0.0, 0.0, 0.06]),
    ]
    for i in range(len(tf)):
        name = f"branch{i}"
        branch = plant.AddRigidBody(name, model_instance, SpatialInertia.SolidCapsuleWithDensity(density, *size, kZ))
        plant.WeldFrames(base.body_frame(), branch.body_frame(), tf[i])
        plant.RegisterVisualGeometry(branch, RigidTransform(), Capsule(*size), name, color)
        plant.RegisterCollisionGeometry(branch, RigidTransform(), Capsule(*size), name, friction)

In [ ]:
class ActuateFilaments:
    def __init__(self, filament_bodies):
        self._filament_bodies = filament_bodies
        self._der_undeformed_states = []
        for body in self._filament_bodies:
            der_model = body.mutable_der_model()
            self._der_undeformed_states.append(der_model.mutable_undeformed_state())
        np.random.seed(0)

    def setRandomTwist(self):
        for undeformed in self._der_undeformed_states:
            m = undeformed.num_internal_nodes()
            undeformed.set_twist(np.random.randn(m) / m)

    def setRandomCurvature(self, extend):
        if not hasattr(self, 'curvature_data'):
            self.curvature_data = []
            for undeformed in self._der_undeformed_states:
                m = undeformed.num_internal_nodes()
                max_angle = np.random.randn(m) * (0.5 * np.pi / m) + (10 * np.pi / m)
                theta = np.random.rand() * (2 * np.pi)
                self.curvature_data.append([max_angle, theta])

        for i, undeformed in enumerate(self._der_undeformed_states):
            max_angle, theta = self.curvature_data[i]
            angle = max_angle * extend
            undeformed.set_curvature_kappa(angle * np.cos(theta), angle * np.sin(theta))

In [ ]:
builder = DiagramBuilder()

plant_config = MultibodyPlantConfig()
plant_config.time_step = 1e-3

scene_graph_config = SceneGraphConfig()
scene_graph_config.default_proximity_properties.compliance_type = "compliant"

plant, scene_graph = AddMultibodyPlant(plant_config, scene_graph_config, builder)

# Add ground
ground = plant.AddRigidBody("ground")
plant.WeldFrames(plant.world_frame(), ground.body_frame(), RigidTransform([0, 0, -0.1]))
plant.RegisterVisualGeometry(
    ground, RigidTransform(), Box(10, 10, 0.2), "ground", [0.7, 0.7, 0.7, 1])
plant.RegisterCollisionGeometry(
    ground, RigidTransform(), Box(10, 10, 0.2), "ground", CoulombFriction(0.1,0.1))

# Add robotic arm
arm_model_instance = Parser(plant, scene_graph).AddModelsFromUrl(
    "package://drake_models/franka_description/urdf/panda_arm.urdf")[0]
plant.WeldFrames(
    plant.world_frame(),
    plant.GetRigidBodyByName("panda_link0").body_frame(),
    RigidTransform()
)
plant.set_gravity_enabled(arm_model_instance, False)
for actuator_index in plant.GetJointActuatorIndices():
    plant.get_mutable_joint_actuator(actuator_index).set_controller_gains(
        PdControllerGains(p=400, d=20))

# Add workpiece
AddWorkpiece(plant)

# Add tool
tool_model_instance = plant.AddModelInstance("tool")
tool = plant.AddRigidBody(
    "flange", tool_model_instance,
    SpatialInertia.SolidCylinderWithDensity(1000, 0.04, 0.02, [0,0,1]))
plant.WeldFrames(
    plant.GetRigidBodyByName("panda_link7").body_frame(),
    tool.body_frame(), RigidTransform([0, 0, 0.11]))
plant.RegisterVisualGeometry(
    tool, RigidTransform(), Cylinder(0.04, 0.02), "flange", [0.2, 0.2, 0.5, 1])
plant.RegisterCollisionGeometry(
    tool, RigidTransform(), Cylinder(0.04, 0.02), "flange", CoulombFriction(0.1,0.1))

# Add filaments for tool
deformable_model = plant.mutable_deformable_model()
filament_bodies = []

for i in range(12):
    filament = Filament(False, np.array([[0,0,0],[0,0,0.26]]).T,
        Filament.CircularCrossSection(diameter=4.2e-3))
    geometry = GeometryInstance(RigidTransform(), filament, f"filament_{i}")

    illus_props = IllustrationProperties()
    illus_props.AddProperty("phong", "diffuse", Rgba(0.7, 0.5, 0.4, 1.0))
    geometry.set_illustration_properties(illus_props)

    proximity_props = ProximityProperties()
    AddContactMaterial(proximity_props, None, None, CoulombFriction(0.4,0.4))
    geometry.set_proximity_properties(proximity_props)

    config = DeformableBodyConfig()
    config.set_youngs_modulus(6.2e5)
    config.set_poissons_ratio(0.4999)
    config.set_mass_density(1155)
    config.set_mass_damping_coefficient(1.1)
    resolution_hint = 0.002

    filament_body_id = deformable_model.RegisterDeformableBody(
        geometry, tool_model_instance, config, resolution_hint)
    filament_bodies.append(deformable_model.GetMutableBody(filament_body_id))

# Set the default arm configuration
plant.Finalize()
q0 = [0, -np.pi*0.4, 0, -np.pi*0.9, 0, np.pi*0.5, 0]
plant.SetDefaultPositions(arm_model_instance, q0)

X_WT = tool.body_frame().CalcPoseInWorld(plant.CreateDefaultContext())  # Pose of tool
for i in range(len(filament_bodies)):
    if i < 8:
        theta = 2 * np.pi / 8 * i
        r = 25e-3
    else:
        theta = 2 * np.pi / 4 * (i - 8)
        r = 12.5e-3
    X_WF = X_WT @ RigidTransform(
        [r * np.cos(theta), r * np.sin(theta), 0.01])  # Pose of filament
    filament_bodies[i].set_default_pose(X_WF)
    filament_bodies[i].AddFixedConstraint(
        tool, X_WT.inverse() @ X_WF,
        Cylinder(0.04, 0.025), RigidTransform()
    )

# Add differential inverse kinematics controller
controller = builder.AddSystem(PoseController())
builder.Connect(
    controller.get_output_port(),
    plant.get_desired_state_input_port(arm_model_instance)
)
builder.Connect(
    plant.get_state_output_port(arm_model_instance),
    controller.get_input_port(1)
)

# Add trajectory source
X_WE_init = plant.GetRigidBodyByName("panda_link7").body_frame().CalcPoseInWorld(
    plant.CreateDefaultContext())
R_WE_init = X_WE_init.rotation()
p_WE_init = X_WE_init.translation()

time = [0, 2, 3, 5, 5.5, 7, 8, 10, 12, 14]
p_WE = np.zeros((3, len(time)))
p_WE = [
    p_WE_init,
    [0.5, 0, 0.6],
    [0.5, 0, 0.6],
    [0.5, 0, 0.39],
    [0.5, 0, 0.33],
    [0.5, 0, 0.33],
    [0.5, 0, 0.36],
    [0.5, 0.1, 0.34],
    [0.5, 0.1, 0.33],
    [0.5, 0.1, 0.6],
]
traj = PiecewisePose(
    PiecewisePolynomial.FirstOrderHold(time, np.array(p_WE).T),
    PiecewiseQuaternionSlerp(time, [R_WE_init] * len(time))
)
source = builder.AddSystem(PoseTrajectorySource(traj))
builder.Connect(source.get_output_port(), controller.get_input_port(0))

# Add Meshcat visualizer
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, MeshcatVisualizerParams()
)

# Helper class to modify the natural curvature of filaments
actuate_filaments = ActuateFilaments(filament_bodies)
actuate_filaments.setRandomTwist()

# Build and simulate
diagram = builder.Build()
simulator = Simulator(diagram)

context = simulator.get_mutable_context()
plant_context = diagram.GetSubsystemContext(plant, context)

simulator.Initialize()
simulator.set_target_realtime_rate(1.0)

visualizer.StartRecording()
simulator.AdvanceTo(5.0)

for t in np.arange(5.0, 7.0, plant_config.time_step):
    simulator.AdvanceTo(t)
    actuate_filaments.setRandomCurvature((t - 5.0) / 2.0)

simulator.AdvanceTo(10.0)

for t in np.arange(10.0, 12.0, plant_config.time_step):
    simulator.AdvanceTo(t)
    actuate_filaments.setRandomCurvature((12.0 - t) / 2.0)

simulator.AdvanceTo(14.0)

visualizer.StopRecording()
visualizer.PublishRecording()